### Import Libraries

In [1]:
# import libraries and set max column width as 100 to display 

import plotly.plotly as py
import pandas as pd
import os

pd.set_option("display.max_columns", 100)

### Get Data

In [2]:
# Importing Data and display fill 

data_h1 = pd.read_csv("H-1B_Disclosure_Data_FY17.csv", low_memory=False)
print("%s MB" %(os.path.getsize('H-1B_Disclosure_Data_FY17.csv') / 1000000))

251.614924 MB


### Data Exploration

In [3]:
# display all column Names

print("Number of features: %d\n" %len(data_h1.columns))
print("##### All features ######")
for col in data_h1.columns:
    print(col)

Number of features: 53

##### All features ######
Unnamed: 0
CASE_NUMBER
CASE_STATUS
CASE_SUBMITTED
DECISION_DATE
VISA_CLASS
EMPLOYMENT_START_DATE
EMPLOYMENT_END_DATE
EMPLOYER_NAME
EMPLOYER_BUSINESS_DBA
EMPLOYER_ADDRESS
EMPLOYER_CITY
EMPLOYER_STATE
EMPLOYER_POSTAL_CODE
EMPLOYER_COUNTRY
EMPLOYER_PROVINCE
EMPLOYER_PHONE
EMPLOYER_PHONE_EXT
AGENT_REPRESENTING_EMPLOYER
AGENT_ATTORNEY_NAME
AGENT_ATTORNEY_CITY
AGENT_ATTORNEY_STATE
JOB_TITLE
SOC_CODE
SOC_NAME
NAICS_CODE
TOTAL_WORKERS
NEW_EMPLOYMENT
CONTINUED_EMPLOYMENT
CHANGE_PREVIOUS_EMPLOYMENT
NEW_CONCURRENT_EMPLOYMENT
CHANGE_EMPLOYER
AMENDED_PETITION
FULL_TIME_POSITION
PREVAILING_WAGE
PW_UNIT_OF_PAY
PW_WAGE_LEVEL
PW_SOURCE
PW_SOURCE_YEAR
PW_SOURCE_OTHER
WAGE_RATE_OF_PAY_FROM
WAGE_RATE_OF_PAY_TO
WAGE_UNIT_OF_PAY
H1B_DEPENDENT
WILLFUL_VIOLATOR
SUPPORT_H1B
LABOR_CON_AGREE
PUBLIC_DISCLOSURE_LOCATION
WORKSITE_CITY
WORKSITE_COUNTY
WORKSITE_STATE
WORKSITE_POSTAL_CODE
ORIGINAL_CERT_DATE


In [4]:
# Proportions of all available Visa types in the data set 

data_h1.VISA_CLASS.value_counts(normalize = True) * 100

H-1B               97.703354
E-3 Australian      1.946210
H-1B1 Singapore     0.200752
H-1B1 Chile         0.149684
Name: VISA_CLASS, dtype: float64

In [5]:
# Getting rid of non h1b visa and checking top 10 states based on h1b visa counts

data_h1 = data_h1[data_h1["VISA_CLASS"] == "H-1B"]
pd.DataFrame(data_h1.EMPLOYER_STATE.value_counts(normalize = True)).head(10) * 100 

EMPLOYER_STATE
CA       16.706757
TX       14.211370
NJ       12.357157
IL        6.687848
NY        6.622633
MI        4.431365
PA        4.140518
MA        3.757583
MD        3.488364
WA        3.278627

In [148]:
# Most popular job in a state

most_popular_job_title = data_h1[["EMPLOYER_STATE","JOB_TITLE"]]
most_popular_job_title = most_popular_job_title.groupby(["EMPLOYER_STATE","JOB_TITLE"]).size().reset_index()
most_popular_job_title.columns = ['EMPLOYER_STATE', 'JOB_TITLE', "COUNT"]
test = most_popular_job_title.groupby(['EMPLOYER_STATE', 'JOB_TITLE']).agg({'COUNT':sum})
test = test['COUNT'].groupby(level=0, group_keys=False)
res = test.apply(lambda x: x.sort_values(ascending=False).head(1))
res = pd.DataFrame(res).reset_index()

tot = pd.DataFrame(data_h1["EMPLOYER_STATE"].value_counts()).reset_index()
tot.columns = ["EMPLOYER_STATE","TOTAL"]
res = pd.merge(res, tot, on = "EMPLOYER_STATE", how = "left")
res["PERCENTAGE"] = res["COUNT"] * 100 /res["TOTAL"]

In [168]:
res.head()

EMPLOYER_STATE               JOB_TITLE  COUNT  TOTAL  PERCENTAGE
0             AK    MEDICAL TECHNOLOGIST      6     75    8.000000
1             AL     ASSISTANT PROFESSOR    144   1174   12.265758
2             AR      PROGRAMMER ANALYST    228   2173   10.492407
3             AS  ASSOCIATE VETERINARIAN      1      1  100.000000
4             AZ       SOFTWARE ENGINEER    281   4032    6.969246

In [178]:
# Companies in a particular state Sponsoring


def H1Bcompanies_in_state(state_two_letter, top = None):
    companies_in_ut = data_h1[data_h1["EMPLOYER_STATE"] == state_two_letter][["EMPLOYER_STATE","EMPLOYER_NAME"]]["EMPLOYER_NAME"].value_counts()
#     for c, n in zip(companies_in_ut.index, companies_in_ut):
#         print(c, n)
    if top == None:result = companies_in_ut 
    else:result = companies_in_ut.head(top)
    return result

In [180]:
# Top 10 coompanies in Utah Sponsoring H1-B

H1Bcompanies_in_state("UT", top = 10)

CONNVERTEX TECHNOLOGIES INC.     234
UNIVERSITY OF UTAH               203
OVERSTOCK.COM, INC.              100
INNOVECTURE                       69
IM FLASH TECHNOLOGIES, LLC        57
NITYA SOFTWARE SOLUTIONS, INC     44
FATPIPE TECHNOLOGIES INC.         42
ANCESTRY.COM OPERATIONS, INC.     40
ZB, N.A.                          39
VAREX IMAGING CORPORATION         30
Name: EMPLOYER_NAME, dtype: int64

In [185]:
# Top 10 companies in Georgia Sponsoring H1-B

H1Bcompanies_in_state("GA",top = 10)

EMORY UNIVERSITY                     288
CHARTER GLOBAL, INC.                 286
NIIT TECHNOLOGIES LIMITED            267
SOFTVISION, LLC                      238
PROFICIENT BUSINESS SYSTEMS, INC.    207
SOFTPATH SYSTEM LLC                  182
ADROIX CORP                          178
DEVISE SOFT, INC                     158
IDHASOFT, INC.                       158
VALUELABS, INC                       157
Name: EMPLOYER_NAME, dtype: int64

In [26]:
title = pd.DataFrame(data_h1.JOB_TITLE.value_counts()).reset_index()
title.columns = ["title","count"]

In [183]:
#title[title["title"].str.contains("DATA SCIENCE") | title["title"].str.contains("DATA SCIENTIST")]

In [28]:
data_h1["PW_WAGE_LEVEL"].value_counts(normalize = True)

Level I      0.445380
Level II     0.359911
Level III    0.124173
Level IV     0.070535
Name: PW_WAGE_LEVEL, dtype: float64

In [29]:
data_h1["PW_UNIT_OF_PAY"].value_counts()

unitpay_to_num = {"Year":1, "Hour": 2080, "Month": 12, "Bi-Weekly": 24}

In [31]:
data_h1["MULTIPLIER"] = data_h1["PW_UNIT_OF_PAY"].map(unitpay_to_num)
data_h1["ACTUAL_SALARY"] = data_h1["WAGE_RATE_OF_PAY_FROM"] * data_h1["MULTIPLIER"]

In [32]:
ass_ds = data_h1[data_h1["JOB_TITLE"] == "ASSOCIATE DATA SCIENTIST"][["ACTUAL_SALARY","WAGE_RATE_OF_PAY_FROM","PW_UNIT_OF_PAY","EMPLOYER_NAME","EMPLOYER_CITY","EMPLOYER_STATE"]]

In [33]:
ass_ds = ass_ds.reset_index(drop = True)

In [34]:
viz = ass_ds.sort_values(by = "ACTUAL_SALARY", ascending= False)

In [35]:
#viz = viz.drop("text", axis = 1)
viz["WAGE_RATE_OF_PAY_FROM"] = viz["WAGE_RATE_OF_PAY_FROM"].astype(float)
final_viz = pd.DataFrame(viz.groupby(['EMPLOYER_STATE'])['WAGE_RATE_OF_PAY_FROM'].mean()).reset_index()

### Only Data Science related jobs

In [36]:
data_h1["JOB_TITLE"] = data_h1["JOB_TITLE"].fillna("not available")

In [57]:
# ds_h1 = data_h1[data_h1["JOB_TITLE"].str.contains("DATA SCIENCE") 
#         | data_h1["JOB_TITLE"].str.contains("DATA SCIENTIST") |
#        data_h1["JOB_TITLE"].str.contains("MACHINE LEARNING") ]

ds_h1 = data_h1[(data_h1["JOB_TITLE"] == "SOFTWARE DEVELOPER") ]

In [58]:
ds_h1.sort_values(by = "ACTUAL_SALARY", ascending= False)

Unnamed: 0         CASE_NUMBER          CASE_STATUS CASE_SUBMITTED  \
299597      299597  I-200-17072-793154            CERTIFIED     2017-03-13   
192634      192634  I-200-17053-372569            CERTIFIED     2017-02-22   
150937      150937  I-200-17038-447928            CERTIFIED     2017-02-07   
430606      430606  I-200-17101-024008            CERTIFIED     2017-04-11   
413503      413503  I-200-17086-574687               DENIED     2017-03-29   
427236      427236  I-200-17095-450179               DENIED     2017-04-10   
386532      386532  I-200-17081-086456            CERTIFIED     2017-03-22   
299793      299793  I-200-17072-679488            CERTIFIED     2017-03-13   
137962      137962  I-200-17030-282420               DENIED     2017-02-01   
452422      452422  I-200-17123-635780            WITHDRAWN     2017-05-03   
306020      306020  I-200-17078-279738            WITHDRAWN     2017-03-19   
571764      571764  I-200-17221-299356            CERTIFIED     2017-08-09   
559986      559986  I-200-17212-037087            CERTIFIED     2017-07-31   
609364      609364  I-200-17255-274175            CERTIFIED     2017-09-12   
98639        98639  I-200-17004-705168            CERTIFIED     2017-01-04   
612619      612619  I-200-17257-548222            CERTIFIED     2017-09-14   
331453      331453  I-200-17074-766187            CERTIFIED     2017-03-16   
333134      333134  I-200-17073-267644            CERTIFIED     2017-03-16   
331385      331385  I-200-17073-364222            CERTIFIED     2017-03-16   
358472      358472  I-200-17076-506982            CERTIFIED     2017-03-20   
373736      373736  I-200-17080-251476            CERTIFIED     2017-03-21   
247543      247543  I-200-17065-359111            CERTIFIED     2017-03-06   
609038      609038  I-200-17249-513524            CERTIFIED     2017-09-12   
565855      565855  I-200-17213-427662            CERTIFIED     2017-08-03   
349321      349321  I-200-17075-698968            CERTIFIED     2017-03-17   
130178      130178  I-200-17026-347365            CERTIFIED     2017-01-27   
43520        43520  I-200-16076-655448  CERTIFIED-WITHDRAWN     2016-03-18   
390225      390225  I-200-17076-833863            CERTIFIED     2017-03-23   
174208      174208  I-200-17045-394936            CERTIFIED     2017-02-16   
407322      407322  I-200-17086-677338            CERTIFIED     2017-03-28   
...            ...                 ...                  ...            ...   
395672      395672  I-200-17087-747769            WITHDRAWN     2017-03-28   
302579      302579  I-200-17072-547121            CERTIFIED     2017-03-13   
229169      229169  I-200-17061-822230            CERTIFIED     2017-03-02   
228398      228398  I-200-17061-788428            CERTIFIED     2017-03-02   
427625      427625  I-200-17087-219911  CERTIFIED-WITHDRAWN     2017-03-28   
251315      251315  I-200-17066-118444            CERTIFIED     2017-03-07   
304036      304036  I-200-17072-612774            CERTIFIED     2017-03-13   
319024      319024  I-200-17074-396217            CERTIFIED     2017-03-15   
301833      301833  I-200-17069-644737            CERTIFIED     2017-03-13   
320812      320812  I-200-17072-189930            CERTIFIED     2017-03-15   
368403      368403  I-200-17070-238688            CERTIFIED     2017-03-21   
403837      403837  I-200-17080-832737            CERTIFIED     2017-03-27   
270295      270295  I-200-17059-183272            CERTIFIED     2017-03-09   
273941      273941  I-200-17060-537123            CERTIFIED     2017-03-09   
336598      336598  I-200-17075-740431            CERTIFIED     2017-03-16   
383243      383243  I-200-17081-910270               DENIED     2017-03-22   
150150      150150  I-200-17038-589939            CERTIFIED     2017-02-07   
415317      415317  I-200-17089-491077            CERTIFIED     2017-03-30   
13281        13281  I-200-16281-872026            CERTIFIED     2016-10-07   
305508      305508  I-200-1707

In [59]:
all_final_viz = pd.DataFrame(ds_h1.groupby(['EMPLOYER_STATE'])['ACTUAL_SALARY'].mean()).reset_index()

#### PLOTLY

In [60]:
df = all_final_viz
for col in df.columns:
    df[col] = df[col].astype(str)

scl = [[0.0, 'rgb(242,240,247)'],[0.2, 'rgb(218,218,235)'],[0.4, 'rgb(188,189,220)'],\
            [0.6, 'rgb(158,154,200)'],[0.8, 'rgb(117,107,177)'],[1.0, 'rgb(84,39,143)']]

df['text'] = df['EMPLOYER_STATE'] + '<br>' +'Salary '+ df['ACTUAL_SALARY']

data = [ dict(
        type='choropleth',
        colorscale = scl,
        autocolorscale = False,
        locations = df['EMPLOYER_STATE'],
        z = df['ACTUAL_SALARY'].astype(float),
        locationmode = 'USA-states',
        text = df['text'],
        marker = dict(
            line = dict (
                color = 'rgb(255,255,255)',
                width = 2
            ) ),
        colorbar = dict(
            title = "Millions USD")
        ) ]

layout = dict(
        title = 'H1 B Heat Map',
        geo = dict(
            scope='usa',
            projection=dict( type='albers usa' ),
            showlakes = True,
            lakecolor = 'rgb(255, 255, 255)'),
             )
    
fig = dict( data=data, layout=layout )
py.iplot( fig, filename='d3-cloropleth-map' )

High five! You successfully sent some data to your account on plotly. View your plot in your browser at https://plot.ly/~jonamjar/0 or inside your plot.ly account where it is named 'd3-cloropleth-map'


In [61]:
test = data_h1[data_h1["EMPLOYER_NAME"] == "OVERSTOCK.COM, INC."]
test[test["JOB_TITLE"] =="ASSOCIATE DATA SCIENTIST"].T

210109
Unnamed: 0                                         210109
CASE_NUMBER                            I-200-17052-877782
CASE_STATUS                                     CERTIFIED
CASE_SUBMITTED                                 2017-02-27
DECISION_DATE                                  2017-03-03
VISA_CLASS                                           H-1B
EMPLOYMENT_START_DATE                          2017-08-21
EMPLOYMENT_END_DATE                            2020-08-20
EMPLOYER_NAME                         OVERSTOCK.COM, INC.
EMPLOYER_BUSINESS_DBA                                 NaN
EMPLOYER_ADDRESS                    799 WEST COLISEUM WAY
EMPLOYER_CITY                                     MIDVALE
EMPLOYER_STATE                                         UT
EMPLOYER_POSTAL_CODE                                84047
EMPLOYER_COUNTRY                 UNITED STATES OF AMERICA
EMPLOYER_PROVINCE                                     NaN
EMPLOYER_PHONE                                 8019473237
EMPLOYER_PHONE_EXT                                    NaN
AGENT_REPRESENTING_EMPLOYER                             Y
AGENT_ATTORNEY_NAME                        PALDINO, FRANK
AGENT_ATTORNEY_CITY                              NEW YORK
AGENT_ATTORNEY_STATE                                   NY
JOB_TITLE                        ASSOCIATE DATA SCIENTIST
SOC_CODE                                          15-2031
SOC_NAME                     OPERATIONS RESEARCH ANALYSTS
NAICS_CODE                                         454111
TOTAL_WORKERS                                           1
NEW_EMPLOYMENT                                          1
CONTINUED_EMPLOYMENT                                    0
CHANGE_PREVIOUS_EMPLOYMENT                              0
NEW_CONCURRENT_EMPLOYMENT                               0
CHANGE_EMPLOYER                                         0
AMENDED_PETITION                                        0
FULL_TIME_POSITION                                      Y
PREVAILING_WAGE                                     44138
PW_UNIT_OF_PAY                                       Year
PW_WAGE_LEVEL                                     Level I
PW_SOURCE                                             OES
PW_SOURCE_YEAR                                       2016
PW_SOURCE_OTHER                   OFLC ONLINE DATA CENTER
WAGE_RATE_OF_PAY_FROM                               65000
WAGE_RATE_OF_PAY_TO                                     0
WAGE_UNIT_OF_PAY                                     Year
H1B_DEPENDENT                                           N
WILLFUL_VIOLATOR                                        N
SUPPORT_H1B                                           NaN
LABOR_CON_AGREE                                       NaN
PUBLIC_DISCLOSURE_LOCATION                            NaN
WORKSITE_CITY                                     MIDVALE
WORKSITE_COUNTY                            SALT LAKE CITY
WORKSITE_STATE                                         UT
WORKSITE_POSTAL_CODE                                84047
ORIGINAL_CERT_DATE                                    NaN
MULTIPLIER                                              1
ACTUAL_SALARY                                       65000

In [55]:
data_h1[data_h1["CHANGE_PREVIOUS_EMPLOYMENT"] != 0]["CASE_STATUS"].value_counts(normalize = True) * 100

CERTIFIED              91.299209
CERTIFIED-WITHDRAWN     4.889081
WITHDRAWN               2.782071
DENIED                  1.029639
Name: CASE_STATUS, dtype: float64

In [56]:
data_h1.columns

Index(['Unnamed: 0', 'CASE_NUMBER', 'CASE_STATUS', 'CASE_SUBMITTED',
       'DECISION_DATE', 'VISA_CLASS', 'EMPLOYMENT_START_DATE',
       'EMPLOYMENT_END_DATE', 'EMPLOYER_NAME', 'EMPLOYER_BUSINESS_DBA',
       'EMPLOYER_ADDRESS', 'EMPLOYER_CITY', 'EMPLOYER_STATE',
       'EMPLOYER_POSTAL_CODE', 'EMPLOYER_COUNTRY', 'EMPLOYER_PROVINCE',
       'EMPLOYER_PHONE', 'EMPLOYER_PHONE_EXT', 'AGENT_REPRESENTING_EMPLOYER',
       'AGENT_ATTORNEY_NAME', 'AGENT_ATTORNEY_CITY', 'AGENT_ATTORNEY_STATE',
       'JOB_TITLE', 'SOC_CODE', 'SOC_NAME', 'NAICS_CODE', 'TOTAL_WORKERS',
       'NEW_EMPLOYMENT', 'CONTINUED_EMPLOYMENT', 'CHANGE_PREVIOUS_EMPLOYMENT',
       'NEW_CONCURRENT_EMPLOYMENT', 'CHANGE_EMPLOYER', 'AMENDED_PETITION',
       'FULL_TIME_POSITION', 'PREVAILING_WAGE', 'PW_UNIT_OF_PAY',
       'PW_WAGE_LEVEL', 'PW_SOURCE', 'PW_SOURCE_YEAR', 'PW_SOURCE_OTHER',
       'WAGE_RATE_OF_PAY_FROM', 'WAGE_RATE_OF_PAY_TO', 'WAGE_UNIT_OF_PAY',
       'H1B_DEPENDENT', 'WILLFUL_VIOLATOR', 'SUPPORT_H1B', 'LAB